In [ ]:
# import requests
# from bs4 import BeautifulSoup
# from datetime import date
# import pandas as pd
# import re
# import os

In [ ]:

# sections= ['news','business','opinion','sports','national','entertainment','photo_feature','feature'
#            ,'world','blog','koseli','diaspora','Education']
# # sections=['news']
# year=2025
# start_month=2
# if (start_month ==12):
#     end_month=1
#     year+=1
# else:
#     end_month=start_month+1


In [ ]:
# def news_exreact(links, df):
#     for link in links:
        
#         response= requests.get(link)
#         soup = BeautifulSoup(response.content, 'html.parser')

#         heading= soup.find('h1').text.strip()
#         heading = re.sub(r'[a-zA-Z0-9/.-]', '', heading)
#         print(heading)
        
#         paragraphs_content = soup.find('div', class_='description current-news-block')
#         try:
#             paragraphs = paragraphs_content.find_all('p')

            
#         except:
#                 print("No paragraphs found")
        
#         all_paragraph = ''
    
#         for paragraph in paragraphs:
#             paragraph= paragraph.text.strip()
#             if(paragraph != '' and paragraph != ' ' and paragraph != '\n'):
#                 paragraph = re.sub(r'[a-zA-Z0-9/.-]', '', paragraph)
#                 paragraph = paragraph.replace('\u2013', '')
#                 all_paragraph=all_paragraph+paragraph +'\n'
                
#         all_paragraph ='"'+ all_paragraph+'"'
#         print(all_paragraph)
        
#         df.loc[len(df)]=[heading, all_paragraph]
        
        
        


# # Define the URL of the page to scrape

In [ ]:
# def change_month(month):
#     if month == 1:
#         return 'Jan'
#     elif month == 2:
#         return 'Feb'
#     elif month == 3:
#         return 'Mar'
#     elif month == 4:
#         return 'Apr'
#     elif month == 5:
#         return 'May'
#     elif month == 6:
#         return 'Jun'
#     elif month == 7:
#         return 'Jul'
#     elif month == 8:
#         return 'Aug'
#     elif month == 9:
#         return 'Sep'
#     elif month == 10:
#         return 'Oct'
#     elif month == 11:
#         return 'Nov'
#     else:
#         return 'Dec'
    

In [ ]:
# def start_extraction(sections, year, start_month, end_month):

#     # sections=['news']

#     for section in sections:
#         # url= f'https://ekantipur.com/search/2025/{section}?date-from=2025-01-01&date-to=2025-02-01'
#         url=f'https://ekantipur.com/search/{year}/{section}?date-from={year}-{start_month}-01&date-to={year}-{end_month}-01'


#         response= requests.get(url)
#         soup = BeautifulSoup(response.content, 'html.parser')
        
#         div_content = soup.find('div', class_='row')
#         # print(div_content)
        
        
#         table_content = div_content.find('div', class_='col-xs-12 col-sm-12 col-md-12')
#         # print(table_content)
#         # print(len(table_content))
        
#         articles_of_table_content=table_content.find_all('div', class_='teaser offset')
#         # print((articles_of_table_content))
        
#         links=[]
#         BASE_URL='https://ekantipur.com'
#         for article in articles_of_table_content:
#         # headline_Link=BASE_URL+((article.find('h2')).find('a')['href'])
#         # link= headline.find('a')['href']
        
#             headline_link= article.find('a')['href']
#             if not headline_link.startswith('http'):
#                 headline_link = BASE_URL + headline_link
        
#                 links.append(headline_link)
        


        
#         blocksLayout= div_content.find('div',class_='blocksLayout')
#         # print(blocksLayout)
    
#         articles_of_blockLayout=blocksLayout.find_all("article")
#         # print((articles_of_blockLayout[0]))
        
#         for article in articles_of_blockLayout:
#             headline_link = article.find('a')['href']
#             if not headline_link.startswith('http'):
#                 headline_link = BASE_URL + headline_link
        
#                 links.append(headline_link)
            
#             # print(headline_link)
#             links.append(headline_link)
            
#         #remove repeated links
#         links = list(set(links))
#         # print(len(links))

#         data = {
#                 'शीर्षक':[],
#                 'विवरण':[],
#                 }

#         df= pd.DataFrame(data)
        
#         # sentLink= [links[0], links[1]]
        
#         news_exreact(links, df)
        
        
        
#         # today = date.today().strftime("%Y-%m-%d") 

#         foldername='/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/ekantipur_past_news/'
        
#         start= change_month(start_month)
#         end= change_month(end_month)

#         file_path = f"{foldername}{section}/{year}-{start}-to-{end} _ekantipur_news.csv"
#         if not os.path.exists(os.path.dirname(file_path)):
#             os.makedirs(os.path.dirname(file_path))

#         if os.path.exists(file_path):
#                 df.to_csv(file_path, mode='a', index=False, header=False)
#         else:
#             df.to_csv(file_path, index=False)

In [ ]:
# start_extraction(sections,year, start_month, end_month)

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import time
import logging
from datetime import date
from urllib.parse import urljoin, urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('news_scraper.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def create_session_with_retry():
    """Create a requests session with retry strategy"""
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })
    return session

def clean_text(text):
    """Clean and normalize text content"""
    if not text:
        return ""
    
    # Remove unwanted characters but keep Nepali text
    text = re.sub(r'[^\u0900-\u097F\s\w\.,!?;:()\-\'\"]+', '', text)
    text = text.replace('\u2013', ' ')  # Replace en-dash
    text = text.replace('\u2014', ' ')  # Replace em-dash
    text = re.sub(r'\s+', ' ', text)    # Replace multiple spaces with single space
    text = re.sub(r'[a-zA-Z0-9/.-]', '', text)
    
    return text.strip()

def news_extract(links, df, session, max_retries=3):
    """Extract news content from links with improved error handling"""
    successful_extractions = 0
    failed_extractions = 0
    
    for i, link in enumerate(links, 1):
        logger.info(f"Processing article {i}/{len(links)}: {link}")
        
        retry_count = 0
        while retry_count < max_retries:
            try:
                # Add delay to be respectful to the server
                time.sleep(1)
                
                response = session.get(link, timeout=10)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract heading with better error handling
                heading_element = soup.find('h1')
                if heading_element:
                    heading = clean_text(heading_element.get_text())
                else:
                    logger.warning(f"No heading found for {link}")
                    heading = "शीर्षक उपलब्ध छैन"

                # Try multiple selectors for content
                content_selectors = [
                    'div.description.current-news-block',
                    'div.description',
                    'div.news-content',
                    'div.content',
                    'article'
                ]
                
                paragraphs = []
                paragraphs_content = None
                
                for selector in content_selectors:
                    paragraphs_content = soup.select_one(selector)
                    if paragraphs_content:
                        paragraphs = paragraphs_content.find_all('p')
                        if paragraphs:
                            break
                
                if not paragraphs:
                    logger.warning(f"No paragraphs found for {link}")
                    all_paragraph = "सामग्री उपलब्ध छैन"
                else:
                    paragraph_texts = []
                    for paragraph in paragraphs:
                        paragraph_text = clean_text(paragraph.get_text())
                        if paragraph_text and len(paragraph_text.strip()) > 0:
                            paragraph_texts.append(paragraph_text)
                    
                    all_paragraph = '\n'.join(paragraph_texts) if paragraph_texts else "सामग्री उपलब्ध छैन"
                
                # Add to dataframe
                new_row = pd.DataFrame({
                    'शीर्षक': [heading],
                    'विवरण': [all_paragraph],
                   
                })
                df = pd.concat([df, new_row], ignore_index=True)
                
                successful_extractions += 1
                logger.info(f"Successfully extracted: {heading[:50]}...")
                break  # Success, exit retry loop
                
            except requests.exceptions.RequestException as e:
                retry_count += 1
                logger.error(f"Request error for {link} (attempt {retry_count}/{max_retries}): {str(e)}")
                if retry_count < max_retries:
                    time.sleep(2 ** retry_count)  # Exponential backoff
                else:
                    failed_extractions += 1
                    logger.error(f"Failed to extract after {max_retries} attempts: {link}")
                    
            except Exception as e:
                retry_count += 1
                logger.error(f"Unexpected error for {link} (attempt {retry_count}/{max_retries}): {str(e)}")
                if retry_count >= max_retries:
                    failed_extractions += 1
    
    logger.info(f"Extraction complete. Success: {successful_extractions}, Failed: {failed_extractions}")
    return df

def validate_inputs(sections, year, start_month, end_month):
    """Validate input parameters"""
    if not sections or not isinstance(sections, list):
        raise ValueError("Sections must be a non-empty list")
    
    if not isinstance(year, int) or year < 2000 or year > 2030:
        raise ValueError("Year must be a valid integer between 2000 and 2030")
    
    if not isinstance(start_month, int) or start_month < 1 or start_month > 12:
        raise ValueError("Start month must be between 1 and 12")
    
    if not isinstance(end_month, int) or end_month < 1 or end_month > 12:
        raise ValueError("End month must be between 1 and 12")
    
    if start_month > end_month:
        raise ValueError("Start month cannot be greater than end month")

def change_month(month_num):
    """Convert month number to month name (assuming this function exists)"""
    months = {
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
        5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
        9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }
    return months.get(month_num, str(month_num).zfill(2))

def extract_links_from_page(soup, base_url):
    """Extract article links from the search results page"""
    links = []
    
    try:
        # Find the main content div
        div_content = soup.find('div', class_='row')
        if not div_content:
            logger.warning("Main content div not found")
            return links
        
        # Extract links from table content
        table_content = div_content.find('div', class_='col-xs-12 col-sm-12 col-md-12')
        if table_content:
            articles_of_table_content = table_content.find_all('div', class_='teaser offset')
            for article in articles_of_table_content:
                try:
                    link_element = article.find('a')
                    if link_element and link_element.get('href'):
                        headline_link = urljoin(base_url, link_element['href'])
                        links.append(headline_link)
                except Exception as e:
                    logger.error(f"Error extracting link from table content: {str(e)}")
        
        # Extract links from blocks layout
        blocks_layout = div_content.find('div', class_='blocksLayout')
        if blocks_layout:
            articles_of_block_layout = blocks_layout.find_all("article")
            for article in articles_of_block_layout:
                try:
                    link_element = article.find('a')
                    if link_element and link_element.get('href'):
                        headline_link = urljoin(base_url, link_element['href'])
                        links.append(headline_link)
                except Exception as e:
                    logger.error(f"Error extracting link from blocks layout: {str(e)}")
        
        # Remove duplicates while preserving order
        links = list(dict.fromkeys(links))
        logger.info(f"Found {len(links)} unique article links")
        
    except Exception as e:
        logger.error(f"Error extracting links from page: {str(e)}")
    
    return links

def start_extraction(sections, year, start_month, end_month, base_folder='/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/ekantipur_past_news/'):
    """Main extraction function with comprehensive error handling"""
    
    try:
        # Validate inputs
        validate_inputs(sections, year, start_month, end_month)
        
        # Create session with retry strategy
        session = create_session_with_retry()
        base_url = 'https://ekantipur.com'
        
        logger.info(f"Starting extraction for {len(sections)} sections from {year}-{start_month:02d} to {year}-{end_month:02d}")
        
        for section in sections:
            logger.info(f"Processing section: {section}")
            
            try:
                # Construct URL
                url = f'{base_url}/search/{year}/{section}?date-from={year}-{start_month:02d}-01&date-to={year}-{end_month:02d}-01'
                logger.info(f"Scraping URL: {url}")
                
                # Get the search results page
                response = session.get(url, timeout=15)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Extract article links
                links = extract_links_from_page(soup, base_url)
                
                if not links:
                    logger.warning(f"No links found for section {section}")
                    continue
                
                # Initialize dataframe
                df = pd.DataFrame({
                    'शीर्षक': [],
                    'विवरण': [],
                    # 'लिंक': []
                })
                
                # Extract news content
                df = news_extract(links, df, session)
                
                if df.empty:
                    logger.warning(f"No data extracted for section {section}")
                    continue
                
                # Prepare file path
                start_month_name = change_month(start_month)
                end_month_name = change_month(end_month)
                
                file_path = os.path.join(
                    base_folder,
            
                    section,
                    f"{year}-{start_month_name}-to-{end_month_name}_ekantipur_news.csv"
                )
                
                # Create directory if it doesn't exist
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                
                # Save to CSV
                if os.path.exists(file_path):
                    df.to_csv(file_path, mode='a', index=False, header=False, encoding='utf-8')
                    logger.info(f"Appended {len(df)} articles to existing file: {file_path}")
                else:
                    df.to_csv(file_path, index=False, encoding='utf-8')
                    logger.info(f"Created new file with {len(df)} articles: {file_path}")
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Network error for section {section}: {str(e)}")
                continue
                
            except Exception as e:
                logger.error(f"Error processing section {section}: {str(e)}")
                continue
                
            # Add delay between sections
            time.sleep(2)
        
        logger.info("Extraction process completed")
        
    except Exception as e:
        logger.error(f"Fatal error in start_extraction: {str(e)}")
        raise

# Example usage
if __name__ == "__main__":
    try:
        sections= ['news','business','opinion','sports','national','entertainment','photo_feature','feature'
           ,'world','blog','koseli','diaspora','Education'] # Add more sections as needed
        start_extraction(sections, 2025, 1,6)
        # url='https://ekantipur.com/search/2014/news?date-from=2014-01-01&date-to=2014-12-31'
    except Exception as e:
        logger.error(f"Script execution failed: {str(e)}")

2025-06-26 08:12:55,414 - INFO - Starting extraction for 13 sections from 2025-01 to 2025-06
2025-06-26 08:12:55,418 - INFO - Processing section: news
2025-06-26 08:12:55,419 - INFO - Scraping URL: https://ekantipur.com/search/2025/news?date-from=2025-01-01&date-to=2025-06-01
2025-06-26 08:12:57,560 - INFO - Found 84 unique article links
2025-06-26 08:12:57,561 - INFO - Processing article 1/84: https://ekantipur.com/news/2025/06/19/dismissal-of-the-order-issued-by-the-district-court-kathmandu-to-remove-the-published-news-08-47.html
2025-06-26 08:13:01,546 - INFO - Successfully extracted: प्रकाशित समाचार हटाउन भन्दै जिल्ला अदालत काठमाडौंल...
2025-06-26 08:13:01,547 - INFO - Processing article 2/84: https://ekantipur.com/news/2025/06/12/i-am-shocked-by-the-news-of-the-air-india-accident-president-paudel-06-57.html
2025-06-26 08:13:05,787 - INFO - Successfully extracted: एयर इन्डियाको दुर्घटनाको खबरले स्तब्ध छु : राष्ट्र...
2025-06-26 08:13:05,788 - INFO - Processing article 3/84: https:/